In [ ]:
import cv2
import os
from pathlib import Path

# === Step 1: Set the base input and output directories ===
input_root = Path("/Users/jasonx/Documents/UTS/25S1/42028 Deep Learning/A3/Part D/dataset").expanduser()
output_root = Path("/Users/jasonx/Documents/UTS/25S1/42028 Deep Learning/A3/Part D/framed_dataset").expanduser()

# === Step 2: Find all .avi files recursively ===
avi_files = list(input_root.rglob("*.avi"))

print(f" Found {len(avi_files)} .avi videos.")

for video_path in avi_files:
    # Get relative path from input_root to video
    relative_path = video_path.relative_to(input_root)

    # Determine where to store the framed video
    output_folder = output_root / relative_path.parent
    output_folder.mkdir(parents=True, exist_ok=True)

    video_name = video_path.stem  # filename without extension
    output_filename = f"{video_name}_framed.mp4"
    output_path = output_folder / output_filename

    print(f"Processing {relative_path} → {output_path.name}")

    # Open and read the video
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        print(f" Could not open: {video_path}")
        continue

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))

    frame_number = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        text = f"Frame: {frame_number}"
        position = (10, height - 10)
        cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX,
                    1, (0, 0, 255), 2, cv2.LINE_AA)

        out.write(frame)
        frame_number += 1

    cap.release()
    out.release()
    print(f"✅ Saved to: {output_path}")

print("🎉 All videos processed successfully.")
